In [4]:
def getInput():
    return open("../input/day_3.txt").read()

def getInputLines():
    return getInput().split("\n")

if only I knew regexps better I could probably use the newlines as boundaries and avoid a lot of checks

# Day 3 Part 1

In [283]:
import re

text = getInput()

# text = """
# 467..114..
# .!.*......
# ..35..633.
# ......#...
# 617*......
# .!...+.58.
# ..592.....
# ......755.
# ...$.*....
# .664.598..
# """.strip()

matches = [match for match in re.finditer(r"\d+", text)]
symbols = set(re.findall(r"\D", re.sub(r"\.", "", re.sub(r"\n", "", text))))

print("Numbers found:", len(matches))
print("Symbols:", symbols)

columns = re.search(r"\n", text).span()[0] # Width
rows = (len(text)-1) // columns

print("Rows:", rows, ", Columns:", columns)

result = 0
for match in matches:
    # Find boundary
    num = match.group()
    width = len(num)
    start, end = match.span()
    row = start // (columns+1) # +1 accounts for the newline
    col = start % (columns+1)
    above, below, left, right = "", "", "", ""
    if row > 0: # check row above
        sAbove = (start - (columns+1)) - (1 if 0 < col < columns else 0)
        eAbove = sAbove + width
        above = text[sAbove:eAbove+2] # add it to the characters to check
    if row < rows: # check row below
        sBelow = (start + (columns+1)) - (1 if 0 < col < columns else 0)
        eBelow = sBelow + width
        below = text[sBelow:eBelow+2]
    if col > 0: # check left
        left = text[start-1]
    if col < columns-1: # check right
        right = text[end]
    if(any([s in (above+left+right+below) for s in symbols])):
        result += int(num)

result

Numbers found: 1227
Symbols: {'#', '&', '*', '+', '=', '@', '%', '/', '$', '-'}
Rows: 140 , Columns: 140


553825

# Day 3 Part 2

In [233]:
# symbols = set(re.findall(r"\D", re.sub(r"\.", "", re.sub(r"\n", "", text))))
# regex_symbolsNoAsterisk = re.sub("\*", "", f"[{''.join(symbols)}]")
# print("Symbols:", symbols)

In [243]:
import re

text = getInput()

# text = """
# 467..114..
# ...*......
# ..35..633.
# ......#...
# 617*......
# .....+.58.
# ..592.....
# ......755.
# ...$.*....
# .664.598..
# """.strip()

asterisks = [match for match in re.finditer("\*", text)]
numbers = [match for match in re.finditer(r"\d+", text)]
nA = len(asterisks)
nN = len(numbers)

columns = re.search(r"\n", text).span()[0] # Width
rows = (len(text)-1) // columns

def roll_index(index:int):
    row = index // (columns+1)
    col = index % (columns+1)
    return row, col

def unroll_index(row:int, col:int):
    return col + (row * columns) + row

print("Rows:", rows, ", Columns:", columns)
print("Asterisks found:", nA)
print("Numbers found:", nN)

#     x   y   z
# *  d1  d2  d3
# *  d4  d5  d6
# *  d9  d8  d7
graph = [[] for _ in range(nA)]
for iA in range(nA):
    ast = asterisks[iA]
    apos = roll_index(ast.span()[0])
    def distance(row:int, col:int):
        d = abs(row-apos[0]) + abs(col-apos[1])
        if abs(row-apos[0]) > 1 or abs(col-apos[1]) > 1 or d > 2:
            return 1000
        return d
    for iN in range(nN):
        num = numbers[iN]
        nums,nume = num.span()
        numspos = roll_index(nums)
        numepos = roll_index(nume-1)
        graph[iA].append((num.group(), distance(*numspos), distance(*numepos)))
    graph[iA] = [node for node in graph[iA] if sum(node[1:]) < 2000]
ratios = [(int(l[0][0]) * int(l[1][0])) for l in graph if len(l) > 1]
sum(ratios)





Rows: 140 , Columns: 140
Asterisks found: 371
Numbers found: 1227


93994191

# Improving my Regex skills

I'm gonna try and re-solve these (pun not intended) making better use of regexps

## Day 3 Part 1 (act 2)

```([.*]?(?<Value>\d+)[.*]?)```


In [279]:
import re

text = getInput()

text = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
""".strip()


# replace all symbols with "#"
text = re.sub(r"[^\d\n\.]", "#", text) 
# find all numbers and the horizontally adjacent symbols
matches = [match for match in re.finditer(r"([\#\.]?(\d+)[\#\.]?)", text)] # .num#, num

text_length = len(text)
line_length = re.search(r"\n", text).start() + 1 # +1 accounting for the \n at each line

print(text)

for match in matches:
    start, width = match.span()
    num_padded, num = match.groups()
    
    # Check line above
    above = start > line_length and re.search(r"\#", text[start-line_length:start-line_length+width]) != None
    below = start < text_length - line_length  and re.search(r"\#", text[start+line_length:start+line_length+width]) != None
    padding = re.search(r"\#",num_padded) != None
    print(num, padding, above, below)

467..114..
...#......
..35..633.
......#...
617#......
.....#.58.
..592.....
......755.
...#.#....
.664.598..
467 False False True
114 False False False
35 False True True
633 False True True
617 True True True
58 False True True
592 False True True
755 False True True
664 False True False
598 False True False


In [296]:
import re

text = getInput()

text = """
467..114..
.!.*......
..35..633.
......#...
617*......
.!...+.58.
..592.....
......755.
...$.*....
.664.598..
""".strip()

# replace all symbols with "#"
text = re.sub(r"[^\d\n\.]", "#", text) 
# find all numbers and the horizontally adjacent symbols
matches = [match for match in re.finditer(r"(\#?(\d+)\#?)", text)] # #num#, num

text_length = len(text)
line_length = re.search(r"\n", text).start() + 1 # +1 accounting for the \n at each line

result = 0
for match in matches:
    start, end = match.span()
    width = end - start
    num_padded, num = match.groups()
    
    # Check line above
    above = start > line_length and re.search(r"\#", text[start-line_length:start-line_length+width]) != None
    padding = re.search(r"\#",num_padded) != None
    below = start < text_length - line_length and re.search(r"\#", text[start+line_length:start+line_length+width]) != None
    if above or padding or below:
        result += int(num)
result
    

467 
.66 None
114 4.598 None
35 #.#. <re.Match object; span=(0, 1), match='#'>
633 ..... None
617 .... None
58 .... None
592 #...# <re.Match object; span=(0, 1), match='#'>
755 ..... None
664 ...#. <re.Match object; span=(3, 4), match='#'>
598 #... <re.Match object; span=(0, 1), match='#'>


4533

In [292]:
matches

[<re.Match object; span=(0, 3), match='467'>,
 <re.Match object; span=(5, 8), match='114'>,
 <re.Match object; span=(24, 26), match='35'>,
 <re.Match object; span=(28, 31), match='633'>,
 <re.Match object; span=(44, 48), match='617#'>,
 <re.Match object; span=(62, 64), match='58'>,
 <re.Match object; span=(68, 71), match='592'>,
 <re.Match object; span=(83, 86), match='755'>,
 <re.Match object; span=(100, 103), match='664'>,
 <re.Match object; span=(104, 107), match='598'>]